In [12]:
if Sys.iswindows()
    username = "Alex"
    pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
    using Pkg
    Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")
else
    username = "sergey"
    pathtorepo = "/home/" *username *"/work/repo/dynamical-systems"
    using Pkg
    Pkg.activate(pathtorepo * "/env/integrate/")
    include(pathtorepo * "/system.jl");
    include("/home/sergey/work/repo/dynamical-systems/FHN/jacobian.jl");
end

  Activating project at `~/work/repo/dynamical-systems/env/integrate`


two_FHN_jac (generic function with 1 method)

In [13]:
using DifferentialEquations, DynamicalSystems, StaticArrays, CairoMakie
using LinearAlgebra, ForwardDiff

In [19]:
params = two_coupled_fhn_get_params();
params[7] = 0.09
params[8] = 100.00
u0 = SVector(-1.0092493342265774, -0.6369086486640424, -1.010765472352876, -0.6365947579973191, 1.6183689231427258);
t_integrate = 1500.0;
tspan = (0.0, t_integrate);
integ_set = (alg = RK4(), adaptive = false, dt = 0.001)

(alg = RK4(; stage_limiter! = trivial_limiter!, step_limiter! = trivial_limiter!, thread = static(false),), adaptive = false, dt = 0.001)

In [20]:
ds = CoupledODEs(two_coupled_fhn, u0, params, diffeq = integ_set)

5-dimensional CoupledODEs
 deterministic: true
 discrete time: false
 in-place:      false
 dynamic rule:  two_coupled_fhn
 ODE solver:    RK4
 ODE kwargs:    (adaptive = false, dt = 0.001)
 parameters:    [0.01, -1.01, 160.0, 0.1, 50.0, 50.0, 0.09, 100.0]
 time:          0.0
 state:         [-1.0092493342265774, -0.6369086486640424, -1.010765472352876, -0.6365947579973191, 1.6183689231427258]


In [21]:
Jacobi(u0) = ForwardDiff.jacobian((x) -> two_coupled_fhn(x, params, 0.0), u0)
Jacobi(u0)

5×5 SMatrix{5, 5, Float64, 25} with indices SOneTo(5)×SOneTo(5):
 -26202.0  -100.0     26178.8    34.0211  -49.0734
      1.0     0.0         0.0     0.0       0.0
  26178.8    33.811  -26202.3  -100.0      49.0734
      0.0     0.0         1.0     0.0       0.0
      1.0     0.0        -1.0     0.0       0.0

In [22]:
det(Jacobi(u0))

-1.918975651231243e-15

In [23]:
two_FHN_jac(u0, params, 0.0)

5×5 SMatrix{5, 5, Float64, 25} with indices SOneTo(5)×SOneTo(5):
 -26202.0  -100.0     26221.6   -34.0211  -49.0734
      1.0     0.0         0.0     0.0       0.0
  26221.5   -33.811  -26202.3  -100.0      49.0734
      0.0     0.0         1.0     0.0       0.0
      1.0     0.0        -1.0     0.0       0.0

In [8]:
det(two_FHN_jac(u0, params, 0.0))

0.0

In [9]:
tands = TangentDynamicalSystem(ds; J = two_FHN_jac)

5-dimensional TangentDynamicalSystem
 deterministic:     true
 discrete time:     false
 in-place:          false
 dynamic rule:      two_coupled_fhn
 jacobian:          two_FHN_jac
 deviation vectors: 5
 parameters:        [0.01, -1.01, 160.0, 0.1, 50.0, 50.0, 0.09, 75.76]
 time:              0.0
 state:             [-1.0092493342265774, -0.6369086486640424, -1.010765472352876, -0.6365947579973191, 1.6183689231427258]


In [10]:
lyapunovspectrum(tands, 20000)

┌ Warning: Instability detected. Aborting
└ @ SciMLBase /home/sergey/.julia/packages/SciMLBase/QSc1r/src/integrator_interface.jl:605


5-element Vector{Float64}:
 103789.20899514909
  96065.88742040105
  16229.710720583276
  12768.667663546907
  -3609.576939025617

In [11]:
lyapunovspectrum(ds, 20000)

┌ Warning: Instability detected. Aborting
└ @ SciMLBase /home/sergey/.julia/packages/SciMLBase/QSc1r/src/integrator_interface.jl:605


5-element Vector{Float64}:
 103788.03118744084
  96206.12165260121
  16160.236651442608
  13307.708359084229
  -3812.309493079699